In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D

# optimizer, data generator and learning rate reductor
from keras.optimizers import Adam

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):                   #on_epoch_end is called when an epoch ends, if the name of function is changed then it won't work
        print("log: " + str(logs))
        if(logs.get('loss')<0.0001):
            #print("\nReached 80% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()


train = pd.read_csv("/Users/vaibhav/Downloads/train.csv")
trainy = train['label']
trainx = train.drop("label", axis=1)

testx = pd.read_csv("/Users/vaibhav/Downloads/test.csv")
    
def create_model():
    model = Sequential()

    model.add(Conv2D(64, kernel_size = 3, activation='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=tf.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate__model(model, trainX, trainY, testX):
    model.fit(trainX, trainY,epochs=20, callbacks = [callbacks], batch_size=64)
    #print(model.evaluate(x=testX, y=testY))
    predictions = model.predict(testX)
    

trainx = trainx.values.reshape(42000,28,28,1)           #for convolution layer(it expects everything in single tensor[4-d] and not 60000 images as a list in dense layer)
testx = testx.values.reshape(28000,28,28,1)             #for convolution layer


#one hot encoder label
trainy = to_categorical(trainy, num_classes = 10)

#trainX, testX, trainY, testY = train_test_split(trainx, trainy, test_size = 0.1)

#normalisation
trainx = np.array(trainx)/255.0
testx = np.array(testx)/255.0
model = None
model = create_model()
train_and_evaluate__model(model, trainx, trainy, testx)

In [ ]:
predictions = model.predict(testx)

In [ ]:
predictions = np.argmax(predictions,axis = 1)

In [ ]:
ImageId = []
for i in range(1, 28001):
    ImageId.append(i)
df = pd.DataFrame(columns = {"ImageId"}, data = ImageId)
df["Label"] = predictions
df.to_csv("----add location to store the csv file----", index=False)